In [1]:
import pandas as pd
import numpy as np


In [2]:
# Load CSV (all as strings initially to inspect safely)
url = "https://raw.githubusercontent.com/gchandra10/filestorage/refs/heads/main/stock_market.csv"
df = pd.read_csv(url, dtype=str, keep_default_na=False)

In [6]:
# 1. Shape
print("Shape:", df.shape)

Shape: (200, 10)


In [8]:
# 2. Preview first 5 rows
print(df.head(5).to_string(index=False))

Trade Date Ticker Open Price Close Price   Volume                  Sector Validated Currency Exchange      Notes
09/01/2025  AAPL      192.47      191.90 21646536             Technology         na        -   NASDAQ    gap up 
09/01/2025   MSFT     456.51      459.76                       Technology         -       na       na         NA
09/01/2025            151.27      150.45 20405836 Consumer Discretionary         na      usd  NASDAQ           -
09/01/2025      -     139.61      141.17 12795579                       -       Yes              NYSE  gap down 
09/01/2025      -    1190.21     1198.56 16389274         Semiconductors         na      USD     NYSE           


In [9]:
# 3. Column list
print(list(df.columns))

['Trade Date', 'Ticker', 'Open Price', 'Close Price', 'Volume', 'Sector', 'Validated', 'Currency', 'Exchange', 'Notes']


In [10]:
# 4. Quick schema / null summary
summary = []
for c in df.columns:
    col = df[c]
    non_empty = col.apply(lambda x: x.strip() if isinstance(x, str) else x).replace({"": None}).notna().sum()
    unique_vals = col.dropna().unique()[:5].tolist()
    summary.append((c, df[c].dtype, non_empty, unique_vals))
    
print("\nSummary (col name, dtype, non-empty count, sample uniques):")
for row in summary:
    print(row)


Summary (col name, dtype, non-empty count, sample uniques):
('Trade Date', dtype('O'), np.int64(200), ['09/01/2025', '09/02/2025', '09/03/2025', '09/04/2025', '09/05/2025'])
('Ticker', dtype('O'), np.int64(185), ['AAPL ', 'MSFT', '', '-', ' TSLA '])
('Open Price', dtype('O'), np.int64(188), ['192.47', '456.51', '151.27', '139.61', '1190.21'])
('Close Price', dtype('O'), np.int64(191), ['191.90', '459.76', '150.45', '141.17', '1198.56'])
('Volume', dtype('O'), np.int64(188), ['21646536', '', '20405836', '12795579', '16389274'])
('Sector', dtype('O'), np.int64(191), [' Technology ', ' Technology', 'Consumer Discretionary ', '-', ' Semiconductors '])
('Validated', dtype('O'), np.int64(182), ['na', '-', 'Yes', 'NO', ''])
('Currency', dtype('O'), np.int64(166), ['-', 'na', 'usd', '', 'USD'])
('Exchange', dtype('O'), np.int64(189), [' NASDAQ', 'na', ' NASDAQ ', ' NYSE', 'NYSE'])
('Notes', dtype('O'), np.int64(179), [' gap up ', 'NA', '-', ' gap down ', ''])


In [12]:
# 5. Convert headers to snake_case
df.columns = (
    df.columns
        .str.strip()                         # remove leading/trailing spaces
        .str.lower()                         # convert to lowercase
        .str.replace(r"\W+", "_", regex=True)  # replace non-alphanumeric with _
)

print(df.columns)

Index(['trade_date', 'ticker', 'open_price', 'close_price', 'volume', 'sector',
       'validated', 'currency', 'exchange', 'notes'],
      dtype='object')


In [14]:
# 6. Trim whitespace in every string field
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

In [17]:
# 7. Lowercase all string columns
df = df.map(lambda x: x.lower() if isinstance(x, str) else x)
print(df.head())

   trade_date ticker open_price close_price    volume                  sector  \
0  09/01/2025   aapl     192.47      191.90  21646536              technology   
1  09/01/2025   msft     456.51      459.76                        technology   
2  09/01/2025            151.27      150.45  20405836  consumer discretionary   
3  09/01/2025      -     139.61      141.17  12795579                       -   
4  09/01/2025      -    1190.21     1198.56  16389274          semiconductors   

  validated currency exchange     notes  
0        na        -   nasdaq    gap up  
1         -       na       na        na  
2        na      usd   nasdaq         -  
3       yes              nyse  gap down  
4        na      usd     nyse            


In [18]:
# 8.: Replace messy missing values
missing_vals = ["", "na", "n/a", "n.a", "null", "-", "none"]

df = df.replace(missing_vals, np.nan)

print("After Missing Values Standardization:")
print(df.isna().sum())

After Missing Values Standardization:
trade_date       0
ticker          42
open_price      37
close_price     36
volume          44
sector          27
validated       86
currency       145
exchange        36
notes           73
dtype: int64


In [20]:
# 9. Fix date formats
date_cols = [col for col in df.columns if col.endswith("_date")]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], format="%Y-%m-%d", errors='coerce')

print("After Date Format Fix:")
print(df[date_cols].head())


After Date Format Fix:
  trade_date
0 2025-09-01
1 2025-09-01
2 2025-09-01
3 2025-09-01
4 2025-09-01


In [21]:
# Your target schema
target_schema = {
    "trade_date": "date",
    "ticker": "string",
    "sector": "string",
    "open_price": "float",
    "close_price": "float",
    "volume": "int"
}

In [23]:
# Apply schema
for col, dtype in target_schema.items():
    if dtype == "date":
        df[col] = pd.to_datetime(df[col], errors="coerce")
    elif dtype == "string":
        df[col] = df[col].astype("string")
    elif dtype == "float":
        df[col] = pd.to_numeric(df[col], errors="coerce")
    elif dtype == "int":
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")


In [25]:
# Save Parquet file to specific path
output_path = r"C:\Users\Narendra\Downloads\cleaned.parquet"
df.to_parquet(output_path, index=False)

print(f"✔ cleaned.parquet saved at: {output_path}")


✔ cleaned.parquet saved at: C:\Users\Narendra\Downloads\cleaned.parquet


In [26]:
# Load cleaned parquet
input_path = r"C:\Users\Narendra\Downloads\cleaned.parquet"
df = pd.read_parquet(input_path)

print("Loaded shape:", df.shape)

Loaded shape: (200, 10)


In [27]:
# AGGREGATION 1: Daily average close price by ticker
agg1 = (
    df.groupby(["trade_date", "ticker"])["close_price"]
    .mean()
    .reset_index()
    .rename(columns={"close_price": "avg_close_price"})
)

agg1_path = r"C:\Users\Narendra\Downloads\agg1.parquet"
agg1.to_parquet(agg1_path, index=False)
print(f"✔ agg1 saved to {agg1_path}")

✔ agg1 saved to C:\Users\Narendra\Downloads\agg1.parquet


In [28]:
# AGGREGATION 2: Average volume by sector
agg2 = (
    df.groupby("sector")["volume"]
    .mean()
    .reset_index()
    .rename(columns={"volume": "avg_volume"})
)

agg2_path = r"C:\Users\Narendra\Downloads\agg2.parquet"
agg2.to_parquet(agg2_path, index=False)
print(f"✔ agg2 saved to {agg2_path}")

✔ agg2 saved to C:\Users\Narendra\Downloads\agg2.parquet


In [31]:
# AGGREGATION 3: Daily simple return by ticker
df = df.sort_values(["ticker", "trade_date"])

df["daily_return"] = (
    df.groupby("ticker")["close_price"]
      .pct_change(fill_method=None)
)

agg3 = df[["trade_date", "ticker", "daily_return"]].dropna()

agg3_path = r"C:\Users\Narendra\Downloads\agg3.parquet"
agg3.to_parquet(agg3_path, index=False)
print(f"✔ agg3 saved to {agg3_path}")


✔ agg3 saved to C:\Users\Narendra\Downloads\agg3.parquet
